# Exercise 20

## a)

The algorithm will almost only generate clusters based on the feature with the biggest magnitude
because the euklidian distance is calculated using the difference in each feature.
A solution is to scale the data, so that all attributes have similar magnitudes.

## b)

The $k$-$NN$ algorithm is a very simple algorithm. It is called a "lazy learner" because it does not learn some sort of decision function, but memorizes the data instead.
Therefore you have no real "training time" for $k$-$NN$, but the predictions are more resource expensive.

Other classifiers like a random forest have a learning phase which needs a certain amount of time but are faster in the classification itself. Exspacially for high dimensional data with many points $k$-$NN$ gets very slow, while a random forest still performs good.

## c)

In [1]:
import numpy as np
import pandas as pd

# read data
S = pd.read_hdf("NeutrinoMC.hdf5", key = "Signal")
B = pd.read_hdf("NeutrinoMC.hdf5", key = "Background")

In [2]:
import statistics
from statistics import mode

class KNN:
    '''KNN Classifier.

    Attributes
    ----------
    k : int
        Number of neighbors to consider.
    '''
    def __init__(self, k):
        '''Initialization.
        Parameters are stored as member variables/attributes.
        
        Parameters
        ----------
        k : int
            Number of neighbors to consider.
        '''
        self.k = k

    def fit(self, X, y):
        '''Fit routine.
        Training data is stored within object.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Training data.
        y : numpy.array shape=(n_samples)
            Training labels.
        '''
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        '''Prediction routine.
        Predict class association of each sample of X.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Data to classify.
        
        Returns
        -------
        prediction : numpy.array, shape=(n_samples)
            Predictions, containing the predicted label of each sample.
        '''
        prediction = []
        
        for x_test in X: # loop over all test data
            
            distances = np.zeros(len(self.X_train))
            
            for i in range(len(distances)): # Calculate distance to all training data points
                distances[i] = np.linalg.norm(x_test - self.X_train[i])
            #distances = np.linalg.norm(x_test - self.X_train)
            
            indices = np.argsort(distances) # sort for smallest distance
            
            # find most frequent label for k nearest neighbors
            keys = []
            for k in range(self.k):
                keys.append(self.y_train[indices[k]])
                
            prediction.append(mode(keys))    # mode(list) returns most freqeunt item in list
            
        return prediction

## d)

In [3]:
signals = S[["NumberOfHits", "x", "y"]][S["x"].notna() * S["y"].notna() * S["NumberOfHits"].notna()]
background = B[["NumberOfHits", "x", "y"]]
events = pd.concat([signals, background])
labels = ["Signal"]*len(signals)
labels.extend(["Background"]*len(background))

n_signal = 10000
n_background = 20000

X_train = np.array(pd.concat([events[:2500], events[30000:32500]]))
y_train = labels[:2500]
y_train.extend(labels[30000:32500])
y_train = np.array(y_train)

X_test = np.array(pd.concat([events[5000:(5000 + n_signal)], events[40000:(40000 + n_background)]]))
y_test = labels[5000:(5000 + n_signal)]
y_test.extend(labels[40000:(40000 + n_background)])
y_test = np.array(y_test)

In [4]:
# Applying fit and prediction with k = 10
knn10 = KNN(k = 10)

knn10.fit(X_train, y_train)

result = knn10.predict(X_test)

In [13]:
tp = len(y_test[(y_test == result) * (y_test == "Signal")])
tn = len(y_test[(y_test == result) * (y_test == "Background")])
fn = len(y_test[(y_test != result) * (y_test == "Background")])
fp = len(y_test[(y_test != result) * (y_test == "Signal")])

precision = tp/(tp + fp)
recall = tp/(tp + fn)
accuracy = (tp + tn)/(n_signal + n_background)

print("precision: ", precision)
print("recall   : ", recall)
print("accuracy : ", accuracy)

precision:  0.9573
recall   :  0.8282574839937705
accuracy :  0.9196


## e)

In [6]:
X_train_log10 = X_train
X_test_log10 = X_test


X_train_log10[:,0] = np.log10(X_train_log10[:,0])
X_test_log10[:,0] = np.log10(X_test_log10[:,0])

In [7]:
knn10.fit(X_train_log10, y_train)

result2 = knn10.predict(X_test_log10)

In [12]:
tp = len(y_test[(y_test == result2) * (y_test == "Signal")])
tn = len(y_test[(y_test == result2) * (y_test == "Background")])
fn = len(y_test[(y_test != result2) * (y_test == "Background")])
fp = len(y_test[(y_test != result2) * (y_test == "Signal")])

precision = tp/(tp + fp)
recall = tp/(tp + fn)
accuracy = (tp + tn)/(n_signal + n_background)

print("precision: ", precision)
print("recall   : ", recall)
print("accuracy : ", accuracy)

precision:  0.9833
recall   :  0.8617123827885373
accuracy :  0.9418333333333333


## f)

In [9]:
knn20 = KNN(k = 20)

knn20.fit(X_train, y_train)

result3 = knn10.predict(X_test)

In [11]:
tp = len(y_test[(y_test == result3) * (y_test == "Signal")])
tn = len(y_test[(y_test == result3) * (y_test == "Background")])
fn = len(y_test[(y_test != result3) * (y_test == "Background")])
fp = len(y_test[(y_test != result3) * (y_test == "Signal")])

precision = tp/(tp + fp)
recall = tp/(tp + fn)
accuracy = (tp + tn)/(n_signal + n_background)

print("precision: ", precision)
print("recall   : ", recall)
print("accuracy : ", accuracy)

precision:  0.9833
recall   :  0.8617123827885373
accuracy :  0.9418333333333333
